<a href="https://colab.research.google.com/github/fabiancpl/my-gpt/blob/master/my-gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>